In [4]:
# 주의점 : 일반 python에서도 scikit-learn 쓸 수 있는지

import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
# 이 부분을 json 파일을 받아오는 것으로 바꾸어야 함

movies =pd.read_csv('tmdb_5000_movies.csv')

movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count', 'overview']]          

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])

movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

genre_sim = cosine_similarity(genre_mat, genre_mat)

genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]

In [24]:
# 정 안 되면 얘를 db에 싣는다..

genre_sim_sorted_ind

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]])

In [39]:
# weighted_vote 구하기

percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)



In [61]:
def find_sim_movie(df, sorted_ind, movie_id, top_n=10):
    # df에서 id인 행과 그 인덱스
    title_movie = df[df['id'] == movie_id]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 - 기준영화
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]


In [63]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 126186, 10)
similar_movies[['title', 'id']]

,title,id
1599,The Age of Adaline,293863
1597,Southpaw,307081
1596,Sicario,273481
1594,Corpse Bride,3933
1603,Stranger Than Fiction,1262
1607,Midnight Run,9013
1605,The Cabin in the Woods,22970
1600,Secondhand Lions,13156
1610,Hanna,50456
1601,Step Up 3D,41233


In [ ]:
# 얘를 이제 json으로 내보낸다